<a href="https://colab.research.google.com/github/Incredible88/Ezio-/blob/master/Finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get start

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
pip install chardet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import chardet
with open('/content/FOMC_minutes.csv', 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']

In [ ]:
encoding

'Windows-1252'

# Load minutes data

In [ ]:
text = pd.read_csv('/FOMC_minutes.csv',encoding='Windows-1252')

In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Month     382 non-null    int64 
 1   Day       382 non-null    int64 
 2   Year      382 non-null    int64 
 3   date      382 non-null    object
 4   sentence  382 non-null    object
dtypes: int64(3), object(2)
memory usage: 15.0+ KB


# load Finbert 

In [2]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finbert = pipeline("text-classification", model=finbert, tokenizer=tokenizer)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.4 MB/s eta 0:00:00


## Using nltk to split sentences

In [ ]:
import nltk

nltk.download('punkt')
def tokenize_sentences(text):
    return nltk.sent_tokenize(text)

text['sentences'] = text['sentence'].apply(tokenize_sentences)

sentences_list = text['sentences'].sum()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
text.sentence[1]

'The Manager of the System Open Market Account reported on recent developments in foreign exchange markets. There were no open market operations in foreign currencies for the System\'s account in the period since the previous meeting. The Manager also reported on developments in domestic financial markets and on System open market transactions in government securities and federal agency obligations during the period since the previous meeting. By unanimous vote, the Committee ratified these transactions.The information reviewed at this meeting suggested that underlying growth in aggregate demand remained solid, even though the expansion of real GDP was estimated to have slowed in the fourth quarter. Household spending rose smartly, outside of autos, and orders and shipments of nondefense capital goods in the business sector were generally quite strong. Housing markets showed some signs of cooling, but starts and sales remained at high levels. Industrial production posted moderate gains

In [ ]:
len(sentences_list)

26412

# Processing data

In [ ]:
nlp(sentences_list[1])

[{'label': 'Neutral', 'score': 0.9999871253967285}]

In [ ]:
results = []
for sentence in tqdm(sentences_list):
    doc = nlp(sentence)
    label = doc[0]['label']
    score = doc[0]['score']
    results.append({'sentence': sentence, 'label': label, 'score': score})

df = pd.DataFrame(results)

100%|██████████| 26412/26412 [1:40:03<00:00,  4.40it/s]


In [ ]:
df.head()

,sentence,label,score
0,The Manager of the System Open Market Account ...,Neutral,0.999998
1,There were no open market operations in foreig...,Neutral,0.999987
2,The Manager also reported on developments in d...,Neutral,0.999989
3,"By unanimous vote, the Committee ratified thes...",Positive,1.000000
4,"Industrial production rebounded, and employmen...",Positive,0.999996


In [9]:
df.to_csv('FOMC_results_3.csv', index=False)

# Text Simplification (sentiment focus)

If there is a transition such as though and while, the focus will be on another paragraph, and if there is a but, the focus will be on the paragraph containing the but. For example:

# **---**
**Although** some scattered signs of cooling of the housing sector had emerged, the pace of construction activity and sales remained brisk.  
⇓  
the pace of construction activity and sales remained brisk.


---


Starts of new single-family homes dropped back somewhat in October from September's very strong pace, **but** permit issuance remained elevated.   
⇓  
permit issuance remained elevated.




In [153]:
import spacy
nlp = spacy.load("en_core_web_sm")

def sentiment_focus(sentence):
    doc = nlp(sentence)
    focus = ""
    focus_changed = 1
    for sent in doc.sents:
        sent_tokens = [token for token in sent]
        for token in sent_tokens:
            if token.lower_ == "although" or token.lower_ == "while":
                try:
                    comma_index = [token1.i for token1 in doc[token.i:] if token1.text == ','][0]
                except IndexError:
                    try:
                      comma_index = [token1.i for token1 in doc[:token.i] if token1.text == ','][-1]
                    except IndexError:
                      return str(doc).strip(),focus_changed
                    focus = doc[:comma_index].text
                    return str(focus).strip(),focus_changed
                focus = doc[comma_index+1:].text
                return str(focus).strip(),focus_changed

    for index, token in enumerate(doc[:-1]):
        if token.lower_ == "but":
            focus = doc[index + 1:]
            return str(focus).strip(),focus_changed
    focus_changed = 0 
    return str(doc).strip(),focus_changed

In [154]:
sentiment_focus(" Starts of new single-family homes dropped back somewhat in October from September's very strong pace, but permit issuance remained elevated")

('permit issuance remained elevated', 1)

In [148]:
sentiment_focus("In their discussion of major sectors of the economy, meeting participants noted that, while light vehicle sales had slowed in the fall, consumer spending outside the auto sector appeared to have remained vigorous")

'consumer spending outside the auto sector appeared to have remained vigorous'

In [141]:
sentiment_focus('Although some scattered signs of cooling of the housing sector had emerged, the pace of construction activity and sales remained brisk.')

'the pace of construction activity and sales remained brisk.'

In [120]:
sentiment_focus("Growth of liquid deposits remained robust, but small time deposits and retail money market mutual funds again contracted at a rapid pace in response to the low yields on those assets")

'small time deposits and retail money market mutual funds again contracted at a rapid pace in response to the low yields on those assets'

In [122]:
finbert("Growth of liquid deposits remained robust, but small time deposits and retail money market mutual funds again contracted at a rapid pace in response to the low yields on those assets")

[{'label': 'Positive', 'score': 0.9998164772987366}]

In [121]:
finbert("small time deposits and retail money market mutual funds again contracted at a rapid pace in response to the low yields on those assets")

[{'label': 'Negative', 'score': 0.7514410614967346}]

In [118]:
finbert('permit issuance remained elevated')

[{'label': 'Negative', 'score': 0.8672457337379456}]

In [119]:
finbert('the pace of construction activity and sales remained brisk.')

[{'label': 'Positive', 'score': 0.3631136417388916}]

## The same sentence after focus processing

Actually works well

In [105]:
finbert("In their discussion of major sectors of the economy, meeting participants noted that, while light vehicle sales had slowed in the fall, consumer spending outside the auto sector appeared to have remained vigorous")

[{'label': 'Negative', 'score': 0.9846646189689636}]

In [117]:
finbert("consumer spending outside the auto sector appeared to have remained vigorous")

[{'label': 'Positive', 'score': 0.8260715007781982}]

# Processing focus sentiment sentences

In [127]:
df.head()

,sentence,label,score
0,The Manager of the System Open Market Account ...,Neutral,0.999998
1,There were no open market operations in foreig...,Neutral,0.999987
2,The Manager also reported on developments in d...,Neutral,0.999989
3,"By unanimous vote, the Committee ratified thes...",Positive,1.000000
4,"Industrial production rebounded, and employmen...",Positive,0.999996


In [3]:
df = pd.read_csv('/content/FOMC_results_2.csv')

In [156]:
import spacy
from tqdm.auto import tqdm
tqdm.pandas()
df[['sentence_simple', 'focus_changed']] = df['sentence'].progress_apply(sentiment_focus).apply(pd.Series)

df['focus_ornot'] = df['focus_changed'].apply(lambda x: 1 if x else 0)

df.drop('focus_changed', axis=1, inplace=True)

  0%|          | 0/26412 [00:00<?, ?it/s]

In [4]:
df.head()

,sentence,label,score,sentence_simple,focus_ornot
0,The Manager of the System Open Market Account ...,Neutral,0.999998,The Manager of the System Open Market Account ...,0
1,There were no open market operations in foreig...,Neutral,0.999987,There were no open market operations in foreig...,0
2,The Manager also reported on developments in d...,Neutral,0.999989,The Manager also reported on developments in d...,0
3,"By unanimous vote, the Committee ratified thes...",Positive,1.000000,"By unanimous vote, the Committee ratified thes...",0
4,"Industrial production rebounded, and employmen...",Positive,0.999996,"Industrial production rebounded, and employmen...",0


## Predict with Finbert

In [7]:
tqdm.pandas()

df['finbert_result'] = df['sentence_simple'].progress_apply(lambda x: finbert(x))

df['label_new'] = df['finbert_result'].apply(lambda x: x[0]['label'])
df['score_new'] = df['finbert_result'].apply(lambda x: x[0]['score'])

df.drop('finbert_result', axis=1, inplace=True)

In [8]:
df.head()

,sentence,label,score,sentence_simple,focus_ornot,label_new,score_new
0,The Manager of the System Open Market Account ...,Neutral,0.999998,The Manager of the System Open Market Account ...,0,Neutral,0.999998
1,There were no open market operations in foreig...,Neutral,0.999987,There were no open market operations in foreig...,0,Neutral,0.999987
2,The Manager also reported on developments in d...,Neutral,0.999989,The Manager also reported on developments in d...,0,Neutral,0.999989
3,"By unanimous vote, the Committee ratified thes...",Positive,1.000000,"By unanimous vote, the Committee ratified thes...",0,Positive,1.000000
4,"Industrial production rebounded, and employmen...",Positive,0.999996,"Industrial production rebounded, and employmen...",0,Positive,0.999996


## Only select changed sentences

In [10]:
df_change = df.loc[df['focus_ornot'] == 1]

In [12]:
df_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4133 entries, 5 to 26403
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentence         4133 non-null   object 
 1   label            4133 non-null   object 
 2   score            4133 non-null   float64
 3   sentence_simple  4133 non-null   object 
 4   focus_ornot      4133 non-null   int64  
 5   label_new        4133 non-null   object 
 6   score_new        4133 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 258.3+ KB


In [13]:
df_change.head()

,sentence,label,score,sentence_simple,focus_ornot,label_new,score_new
5,Although some scattered signs of cooling of th...,Negative,0.645584,the pace of construction activity and sales re...,1,Positive,0.363114
10,With employment rising but the average workwee...,Negative,0.999218,the average workweek of production or nonsuper...,1,Negative,0.997591
19,Real disposable personal income was bolstered ...,Positive,1.000000,Real disposable personal income was bolstered ...,1,Positive,1.000000
21,The personal saving rate--while still slightly...,Positive,0.997309,The personal saving rate--while still slightly...,1,Positive,0.997309
22,Starts of new single-family homes dropped back...,Negative,0.950385,permit issuance remained elevated.,1,Negative,0.691221


## Select predicted label changed sentences

In [14]:
df_change1 = df_change.loc[df_change.label != df_change.label_new]

In [15]:
df_change1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1208 entries, 5 to 26390
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentence         1208 non-null   object 
 1   label            1208 non-null   object 
 2   score            1208 non-null   float64
 3   sentence_simple  1208 non-null   object 
 4   focus_ornot      1208 non-null   int64  
 5   label_new        1208 non-null   object 
 6   score_new        1208 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 75.5+ KB


In [17]:
df_change1.head()

,sentence,label,score,sentence_simple,focus_ornot,label_new,score_new
5,Although some scattered signs of cooling of th...,Negative,0.645584,the pace of construction activity and sales re...,1,Positive,0.363114
33,Foreign industrialized economies expanded robu...,Positive,1.000000,especially in transportation services.,1,Negative,0.492485
38,Longer-term inflation expectations also edged ...,Positive,0.970137,stayed a touch above the narrow range observed...,1,Neutral,0.946409
46,"In these circumstances, the Committee believed...",Neutral,0.929808,noted that it would respond to changes in econ...,1,Negative,0.516684
55,Although hurricane-related rebuilding would bo...,Positive,0.593083,"especially in the near term, this stimulus inc...",1,Negative,0.998140


In [18]:
df_change1.to_csv("changing_labels.csv",index=False)